<a href="https://colab.research.google.com/github/DANCAR1969/programacion/blob/master/NPL_VOZ_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Paso 1: Instalar dependencias
!pip install transformers
!pip install gTTS
!pip install SpeechRecognition
!pip install pydub
!apt install ffmpeg -y

# Paso 2: Importar librerías
import speech_recognition as sr
from gtts import gTTS
from IPython.display import Audio, display, Markdown
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os

# Paso 3: Cargar modelo
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Paso 4: Contexto universitario
contexto = (
    "Eres un asistente virtual universitario. "
    "Responde con claridad, tono profesional y académico. Brinda información sobre admisiones, matrícula, programas académicos, contacto institucional y vida estudiantil.\n"
    "Estudiante: Hola\n"
    "Asistente:"
)

chat_history_ids = None
step = 0

# Paso 5: Función para hablar
def hablar(texto, nombre="respuesta.mp3"):
    tts = gTTS(text=texto, lang='es')
    tts.save(nombre)
    display(Audio(nombre, autoplay=True))

# Paso 6: Función para reconocer voz desde archivo
def reconocer_audio(file_path):
    r = sr.Recognizer()
    with sr.AudioFile(file_path) as source:
        audio = r.record(source)
    try:
        texto = r.recognize_google(audio, language="es-ES")
        print(f"🗣️ Estudiante dijo: {texto}")
        return texto
    except:
        print("⚠️ No se pudo reconocer el audio.")
        return None

# Paso 7: Función principal del chatbot
def chatbot_universitario(entrada):
    global chat_history_ids, step

    display(Markdown(f"**Estudiante:** {entrada}"))

    if step == 0:
        full_input = contexto + " " + entrada + tokenizer.eos_token
        input_ids = tokenizer.encode(full_input, return_tensors='pt')
    else:
        input_ids = tokenizer.encode(entrada + tokenizer.eos_token, return_tensors='pt')
        input_ids = torch.cat([chat_history_ids, input_ids], dim=-1)

    chat_history_ids = model.generate(input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    respuesta = tokenizer.decode(chat_history_ids[:, input_ids.shape[-1]:][0], skip_special_tokens=True)

    display(Markdown(f"**Asistente:** {respuesta}"))
    hablar(respuesta)
    step += 1

# Paso 8: Subir archivo de audio WAV
from google.colab import files
print("📤 Suba un archivo de audio en formato WAV (máximo 20 segundos, voz clara).")
uploaded = files.upload()

# Paso 9: Procesar archivo
for filename in uploaded.keys():
    texto_usuario = reconocer_audio(filename)
    if texto_usuario:
        chatbot_universitario(texto_usuario)
    else:
        hablar("Lo siento, no pude entender el audio.")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
📤 Suba un archivo de audio en formato WAV (máximo 20 segundos, voz clara).


Saving DLVOZ.wav to DLVOZ (1).wav
🗣️ Estudiante dijo: Hola soy Daniel carrera actualmente soy ejecutor de tecnologías de la información y talento Tec la verdad ha sido una experiencia magnífica ya que he podido desarrollar diferentes proyectos enfocados a Inteligencia artificial más que todo en el área de Machine learning the learning por supuesto también la parte de ciberseguridad y bueno algunas otras


**Estudiante:** Hola soy Daniel carrera actualmente soy ejecutor de tecnologías de la información y talento Tec la verdad ha sido una experiencia magnífica ya que he podido desarrollar diferentes proyectos enfocados a Inteligencia artificial más que todo en el área de Machine learning the learning por supuesto también la parte de ciberseguridad y bueno algunas otras

**Asistente:** I